In [ ]:
# Load packages for data analysis
import getpass, re, json, sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
import pandas_gbq as pgbq
# Load packages for Big Query 
from google.cloud import bigquery
import os
from sqlalchemy import create_engine

In [ ]:
# Define configurations for Big Query
project_id = '' # Location of stride datalake
client = bigquery.Client(project=project_id) # Set project to project_id
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''
os.environ['GCLOUD_PROJECT'] = "" # specify environment
db = "" # Define the database
stanford_ds = ""
yh_ds = ""

In [ ]:
def save_table(project_id, yh_ds, new_table_name, query):
    table_id = f"{project_id}.{yh_ds}.{new_table_name}"
    job_config = bigquery.QueryJobConfig(destination=table_id)
    job_config.write_disposition = "WRITE_TRUNCATE"
    # Start the query, passing in the extra configuration.
    query_job = client.query(query, job_config=job_config)  # Make an API request.
    query_job.result()  # Wait for the job to complete.
    print("Query results loaded to the table {}".format(table_id))  
def load_pgbq(project_id, yh_ds, table_name):
    sql_query = f"SELECT * FROM {project_id}.{yh_ds}.{table_name}"
    return_df = pgbq.read_gbq(sql_query, dialect="standard")
    print (f"{project_id}.{yh_ds}.{table_name}", "is loaded") 
    return return_df
def upload_pgbq(project_id, yh_ds, table_name, df):
    table_id = f"{yh_ds}.{table_name}"
    pgbq.to_gbq(df, table_id, project_id=project_id)
    print ("dataframe", df, "is uploaded as", f"{project_id}.{yh_ds}.{table_name}") 
def remove_table(project_id, yh_ds, table_name):
    client = bigquery.Client()
    table_id = f"{project_id}.{yh_ds}.{table_name}"
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

In [ ]:
comorbidity = pd.read_excel("/Users/yeonmihwang/Documents/Research/Projects/dementia/opioid/resource/codes_ComorbidityMedication.xlsx", 
                        sheet_name = 'comorbidity')
comorbidity.head(10)

In [ ]:
comorbidity['diagnosis'].tolist()

In [ ]:
total_icd10s = []
for c in comorbidity['diagnosis'].tolist():
    total_icd10s += comorbidity[comorbidity['diagnosis']==c]['ICD-10'].iloc[0].replace(" ", "").split(",")

In [ ]:
def save_table(project_id, yh_ds, new_table_name, query):
    table_id = f"{project_id}.{yh_ds}.{new_table_name}"
    job_config = bigquery.QueryJobConfig(destination=table_id)
    job_config.write_disposition = "WRITE_TRUNCATE"
    # Start the query, passing in the extra configuration.
    query_job = client.query(query, job_config=job_config)  # Make an API request.
    query_job.result()  # Wait for the job to complete.
    print("Query results loaded to the table {}".format(table_id))  
def load_pgbq(project_id, yh_ds, table_name):
    sql_query = f"SELECT * FROM {project_id}.{yh_ds}.{table_name}"
    return_df = pgbq.read_gbq(sql_query, dialect="standard")
    print (f"{project_id}.{yh_ds}.{table_name}", "is loaded") 
    return return_df
def upload_pgbq(project_id, yh_ds, table_name, df):
    table_id = f"{yh_ds}.{table_name}"
    pgbq.to_gbq(df, table_id, project_id=project_id)
    print ("dataframe", df, "is uploaded as", f"{project_id}.{yh_ds}.{table_name}") 
def remove_table(project_id, yh_ds, table_name):
    client = bigquery.Client()
    table_id = f"{project_id}.{yh_ds}.{table_name}"
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

In [ ]:
def generate_regex_icd10(ancestor): 
    if '.' in ancestor: 
        split = ancestor.split(".")
        regex_string = f"{split[0]}\\.{split[1]}(\\d+)?"
    else:
        regex_string = f"{ancestor}(\\.\\d+)?"
    return regex_string

def generate_sql_regex_icd10s(icd10_list):
    patterns = [generate_regex_icd10(icd10) for icd10 in icd10_list]
    separator = "|"
    pattern_string = separator.join(patterns)
    regex_string = f"^({pattern_string})$" 
    return regex_string 
icd10_string = generate_sql_regex_icd10s(total_icd10s)

In [ ]:
string  = f"REGEXP_CONTAINS(diag.icd10, r'({icd10_string})')"
string

In [ ]:
# dementia diagnosis codes 
sql_query = f"""
    SELECT 
        pat.pat_deid,
        DATE_DIFF(DATE(diag.start_date), DATE(pat.birth_date), YEAR) AS age_at_comorbid_diagnosis,
        DATE_DIFF(DATE(pat.diagnosis_date), DATE(pat.birth_date), YEAR) AS age_at_dementia,
        pat.sex, 
        pat.ethnic_group, 
        pat.death_date, 
        pat.deceased, 
        pat.race, 
        pat.diagnosis_date AS dementia_start_date,
        diag.dx_id, 
        diag.dx_name, 
        diag.icd10, 
        diag.start_date AS diagnosis_start_date, 
        DATE_DIFF(DATE(pat.diagnosis_date), DATE(diag.start_date), DAY) AS days_before_dementia
    FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_diagnosis` AS diag
    JOIN `som-nero-phi-boussard.YH_dementia.dementia_pat_coverage_07312024` AS pat ON pat.pat_deid = diag.pat_deid
    WHERE (REGEXP_CONTAINS(diag.icd10, r'{icd10_string}')) AND (DATE(diag.start_date) < DATE(pat.diagnosis_date)) AND (DATE(diag.start_date) >= DATE_SUB(DATE(pat.diagnosis_date), INTERVAL 2 YEAR))
"""
#save raw dementia patient table 
table_name = "dementia_pat_comorbidity_07312024"
save_table(project_id, yh_ds, table_name, sql_query)

In [ ]:
# dementia diagnosis codes 
sql_query = f"""
    SELECT 
        pat.pat_deid,
        DATE_DIFF(DATE(diag.start_date), DATE(pat.birth_date), YEAR) AS age_at_comorbid_diagnosis,
        DATE_DIFF(DATE(pat.diagnosis_date), DATE(pat.birth_date), YEAR) AS age_at_dementia,
        pat.sex, 
        pat.ethnic_group, 
        pat.death_date, 
        pat.deceased, 
        pat.race, 
        pat.diagnosis_date AS dementia_start_date,
        diag.dx_id, 
        diag.dx_name, 
        diag.icd10, 
        diag.start_date AS diagnosis_start_date, 
        DATE_DIFF(DATE(pat.diagnosis_date), DATE(diag.start_date), DAY) AS days_before_dementia
    FROM `som-nero-phi-boussard.stanfordmed_datalake.shc_diagnosis` AS diag
    JOIN `som-nero-phi-boussard.YH_dementia.dementia_pat_exposure_group` AS pat ON pat.pat_deid = diag.pat_deid
    WHERE (REGEXP_CONTAINS(diag.icd10, r'{icd10_string}')) AND (DATE(diag.start_date) < DATE(pat.post_onset_exposure_start_time)) AND (DATE(diag.start_date) >= DATE_SUB(DATE(pat.diagnosis_date), INTERVAL 2 YEAR))
"""
#save raw dementia patient table 
table_name = "dementia_pat_comorbidity_before_op_07312024"
save_table(project_id, yh_ds, table_name, sql_query)

In [ ]:
def get_string (comorbidity, comorbidity_df):
    icd_10s = comorbidity_df[comorbidity_df['diagnosis']==comorbidity]['ICD-10'].iloc[0].replace(" ", "").split(",")
    comorbidity = comorbidity.replace(" ","_")
    comorbidity = comorbidity.replace("/","_")
    comorbidity = comorbidity.replace("(","_")
    comorbidity = comorbidity.replace(")","_")
    comorbidity = comorbidity.replace(",","_")
    comorbidity = comorbidity.replace("-","_")
    diagnosis_string = generate_sql_regex_icd10s(icd_10s)
    string = f"CASE WHEN REGEXP_CONTAINS(diag.icd10, r'{diagnosis_string}') THEN 1 ELSE 0 END AS {comorbidity}"
    return string
comorbidity_df = pd.read_excel("/Users/yeonmihwang/Documents/Research/Projects/dementia/opioid/resource/codes_ComorbidityMedication.xlsx", sheet_name = 'comorbidity')

In [ ]:
c_list = comorbidity['diagnosis'].tolist()

In [ ]:
stanford_ds = "stanfordmed_datalake"
yh_ds = "YH_dementia"
sql_table = "dementia_pat_comorbidity_07312024"
sql_query = f""" 
SELECT *,
"""
for c_index in range(len(c_list)):
    c = c_list[c_index]
    sql_query += get_string(c, comorbidity_df)
    if c_index < len(c_list)-1:
        sql_query += """,\n"""
    if c_index == len(c_list)-1:
        sql_query += f"""\nFROM {db}.{yh_ds}.{sql_table} AS diag;"""
print (sql_query)

In [ ]:
table_name = "dementia_comorbidity_categories_07312024"
save_table(project_id, yh_ds, table_name, sql_query)

In [ ]:
table_id = f"{db}.{yh_ds}.{table_name}"
# Fetch the table schema
table = client.get_table(table_id)

# Get the list of column names
column_names = [schema_field.name for schema_field in table.schema]

# Print the column names
print(column_names)

In [ ]:
stanford_ds = "stanfordmed_datalake"
yh_ds = "YH_dementia"
sql_table = "dementia_pat_comorbidity_before_op_07312024"
sql_query = f""" 
SELECT *,
"""
for c_index in range(len(c_list)):
    c = c_list[c_index]
    sql_query += get_string(c, comorbidity_df)
    if c_index < len(c_list)-1:
        sql_query += """,\n"""
    if c_index == len(c_list)-1:
        sql_query += f"""\nFROM {db}.{yh_ds}.{sql_table} AS diag;"""
print (sql_query)

In [ ]:
table_name = "dementia_comorbidity_before_op_categories_07312024"
save_table(project_id, yh_ds, table_name, sql_query)

In [ ]:
table_id = f"{db}.{yh_ds}.{table_name}"
# Fetch the table schema
table = client.get_table(table_id)

# Get the list of column names
column_names = [schema_field.name for schema_field in table.schema]

# Print the column names
print(column_names)

In [ ]:
grouping_columns = ["pat_deid", "age_at_dementia", "sex", "ethnic_group", "death_date", "deceased", "race", "dementia_start_date"]
aggregate_max_columns = ['myocardial_infarction', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease_', 'chronic_pulmonary_disease', 'rheumatologic_disease', 'peptic_ulcer', 'hemiplegia_paraplegia', 'diabetes_without_complications', 'diabetes_with_chronic_complications_', 'mild_liver_diseases', 'moderate_severe_liver_disease', 'renal_disease', 'any_malignancy__tumor__leukemia__lymphoma_', 'metastatic_solid_tumor', 'HIV_AIDS', 'cognitive_decline', 'hypertension', 'hyperlipidemia', 'excessive_alcohol', 'atherosclerosis', 'hypercholesterolemia', 'atrial_fibrillation', 'traumatic_brain_injury', 'hearing_loss', 'sleep_apnea', 'prediabetes', 'delirium', 'depression', 'schizophrenia_non_mood_psychotic_disorder', 'mood_disorder', 'anxiety_and_non_psychotic_mental_disorder', 'behavioral_syndromes_associated_with_physiological_disturbances_and_physical_factors', 'disorders_of_adult_personality_and_behavior', 'Intellecutual_disability', 'developmental_disorder', 'behavioral_and_emotional_isorder_with_onset_in_early_ages']

In [ ]:
sql = """
SELECT"""
for gc in grouping_columns: 
    sql+=f"\n     {gc},"
for mc_index in range(len(aggregate_max_columns)):
    mc = aggregate_max_columns[mc_index]
    if mc_index < len(aggregate_max_columns)-1:
        sql+=f"\n     MAX({mc}) AS {mc},"
    if mc_index == len(aggregate_max_columns)-1:
        sql+=f"\n     MAX({mc}) AS {mc}"
sql+=f"\nFROM\n     {table_id}"
sql+="\nGROUP BY"
for gc_index in range(len(grouping_columns)): 
    gc = grouping_columns[gc_index]
    if gc_index < len(grouping_columns)-1:
        sql+=f"\n     {gc},"
    if gc_index == len(grouping_columns)-1:
        sql+=f"\n     {gc};"
print (sql)

In [ ]:
table_name = "dementia_comorbidity_categories_aggregated_07312024"
save_table(project_id, yh_ds, table_name, sql)

In [ ]:
sql = """
SELECT"""
for gc in grouping_columns: 
    sql+=f"\n     {gc},"
for mc_index in range(len(aggregate_max_columns)):
    mc = aggregate_max_columns[mc_index]
    if mc_index < len(aggregate_max_columns)-1:
        sql+=f"\n     MAX({mc}) AS before_exposure_{mc},"
    if mc_index == len(aggregate_max_columns)-1:
        sql+=f"\n     MAX({mc}) AS before_exposure_{mc}"
sql+=f"\nFROM\n     {table_id}"
sql+="\nGROUP BY"
for gc_index in range(len(grouping_columns)): 
    gc = grouping_columns[gc_index]
    if gc_index < len(grouping_columns)-1:
        sql+=f"\n     {gc},"
    if gc_index == len(grouping_columns)-1:
        sql+=f"\n     {gc};"
print (sql)

In [ ]:
table_name = "dementia_comorbidity_before_op_categories_aggregated_07312024"
save_table(project_id, yh_ds, table_name, sql)